## PIO
### Projekt 5 [50 pkt]

Proszę użyć tego arkusza do programowania. Na początek proszę zmienić 'nazwisko_imie' na Państwa nazwisko i imie, bez polskich znaków diakrytycznych.

Jak już Państwo zaprogramują wszystko, proszę zapisać arkusz (File->Save and Checkpoint) oraz zapisać arkusz na dysk (File->Download as->Notebook). Ten plik proszę wgrać na classroom. 

### Automatyczna klasówka z języka angielskiego

Proszę napisać program generujący automatyczne klasówki z języka angielskiego. Bazujemy na problemie z lab 9, ale treści do klasówek proszę pobierać ze strony interentowej oferującej naukę języka np: https://www.gettinenglish.com

Funkcjonalność 
1. Uczniowie - czytanie listy uczniów z pliku
2. Klasówka
    1. minimum 3 zadania różniące się choć trochę treścią dla każdego z uczniów
    2. zapisywanie treści klasówek do plików imiennych
    3. składowanie plików w katalogu o nazwie tożsamej z nazwą egzaminu (klasówki...)
3. Pobieranie treści z www
    1. konfiguracja adresu strony www powinna być w miarę prosta i zrozumiała
    2. parsowanie strony powinno być choć w niewielkim stopniu dostępne za pomocą parametrów (np: klasy tagów do których się odwołuje parser)

Opcjonalnie
* automatyczne wysyłanie klasówek na adresy mailowe uczniów


Box(children=(VBox(children=(HBox(children=(Button(description='Load student lists', style=ButtonStyle()), But…

Box(children=(VBox(children=(HBox(children=(Button(description='Load student lists', style=ButtonStyle()), But…

In [90]:
import urllib.request as urllib2
from bs4 import BeautifulSoup

def parseGettinEnglish(url, _type='RC', log = False):
    """ 
    gettinenglish.com parser. 
        RC = Reading Comprehension 
    """
    data = urllib2.urlopen(url)
    soup = BeautifulSoup(data, 'html.parser')


    title = ''
    stuff = ''
    answers = ''
    try:
        title = soup.find("div", {"class": "single-post-content"}).h2.extract().getText().strip()
    except:
        title = soup.find("h1", {"class": "single-post-title"}).getText().strip()

    stuff = soup.find("div", {"class": "single-post-content"}).findAll('p')
    
    for n, p in enumerate(stuff):
        if log:
            print(n)  
            print(p.prettify())
            print()
        try:
    #         p.a.replace_with('')
            p.a.extract()
            p.div.extract()
            pass

        except:
            pass

        if (p.getText()=='') or (p.strong is not None):
            stuff.pop(n)

    text = '\n\n'.join((map(BeautifulSoup.getText, stuff)))
    
    print(url)
    print(title) 
    print(text)
    print()
    return (title, text)


url = "https://www.gettinenglish.com/egzaminy/czytanie-ze-zrozumieniem/"

data = urllib2.urlopen(url)
soup = BeautifulSoup(data, 'html.parser')
_urls = [x['href'] for x in soup.find("div", {"class": "main-section"}).findAll("a", {"class": "post-url"})]

for n, _ in enumerate(_urls):
    (title, text) = parseGettinEnglish(url = _,  _type='RC')
    print(n, _)


https://www.gettinenglish.com/materialy-do-nauki-angielskiego/
I wanna stay at home
Agoraphobia is an anxiety disorder in which people may experience fear and frequently refrain from getting into a situation that can make them feel trapped, embarrassed or helpless. Some agoraphobics are fearful of open spaces. Some even in a medical emergency, refuse to leave their homes because stepping outside their comfort zone is overwhelming. An agoraphobic also breaks out in a sweat while standing in a queue or being in a crowd. Most people who have agoraphobia avoid the place where deep emotional distress is likely to occur; they feel that they need a chum to accompany them to any public places.

„My name’s Melissa and I’ve suffered from agoraphobia since my early twenties. I know it sounds outlandish but every time I wanna go out I feel tingling in my hands and feet, nausea, and a sense of impending doom!”

A number of physical and environmental factors can trigger agoraphobia and other phobias

In [16]:
from ipywidgets import *
import validators
import urllib.request as urllib2
from bs4 import BeautifulSoup, NavigableString, Tag
import re 

class GettinParser():  
    _gui = widgets.Output(layout={'border': '1px solid black'})
    _url = None
    _soup = BeautifulSoup()
    _title_filter = ['post-header-title']
    
    
    def display_GUI(self):
#         self._gui  = widgets.Output(layout={'border': '1px solid black'})
        self._gui.clear_output()
        self.updateGUI()
        display(self._gui)
        
        
    @property
    def url(self):
        return self._url
    
    @url.setter
    def url(self, value):
        try:
            if validators.url(value):
                print("Setting value of url to {}".format(value))
                self._url = value
                self.update()
            else:
                print("Error: {} is not a valid url".format(value))
            
        except Exception as e:
            print(f"{type(e).__name__} was raised: {e}") 
    
    def update(self):
        self.parseURL()

    def parseURL(self):
        data = urllib2.urlopen(self._url)
        self._soup = BeautifulSoup(data, 'html.parser')
        self._titleOutput=widgets.Output()
        
        def soupWalker(soup, level=0):
       
            indent = (level) * '·' + 'o'

            if soup.name is not None:
                for child in soup.children:
                    if isinstance(child, Tag):
                        name = child.name
                        classList = (child.get('class'))
                        if classList is not None:
                            classes = ', '.join(list(classList))
                        else:
                            classes = 'none'
                        with self._titleOutput:
                            print(indent, '<{}> ({})'.format(name, classes))

                    soupWalker(child, level+1)
                    
        self._titles = self._soup.find_all(class_=self._title_filter)
        
#         self._titles = self._soup.find_all(class_=lambda c: all(f in c for f in self._title_filter))

        
        for node in (self._titles):
            soupWalker(node.parent)
            
        self.updateGUI()
        
    def updateGUI(self):
        self._gui.clear_output()
        
        buttonLayout={'width': '120px'}
        textAreaLayout={'width': '50%', 'height': '300px'}
        
        urlText = (Text(value=self.url,
                        placeholder='Enter url...', 
                        disabled=False, 
                        continuous_update=False))
        
        def urlButtonCallback(b, value=None):
            self.url = (urlText.value) 
            
        urlText.observe(urlButtonCallback, names='value')
        
        parseText = (Textarea(value=self._soup.prettify(),
                        disabled=False,  
                        layout=textAreaLayout))
        
        titleText = (Text(value=', '.join(self._title_filter),
                          description='Filter classes:', 
                          continuous_update=False))
        
        def on_value_change(change):
            self._title_filter=(change['new']).split(',')
            print(self._title_filter)
            self.update()

        titleText.observe(on_value_change, names='value')
        
        titleDropdown = Dropdown(options=list(map(BeautifulSoup.getText, self._titles)),
                                description='Choose:', 
                                disabled=False)
        
        titleOutput= self._titleOutput
        
        
        app = VBox(children = [urlText,
                              HBox(children = [VBox(children = [titleText, titleDropdown, titleOutput]), 
                                               parseText])]) #titleOutpu
        
        with self._gui:
            display(app)
        

In [17]:
parser = GettinParser()
#parser.url = 'https://www.gettinenglish.com/angielski-tekst-do-czytania-c1/' 'https://www.gettinenglish.com/czytanki-po-angielsku-b1/'
parser.url = 'https://www.gettinenglish.com/czytanki-po-angielsku-b1/' 
parser.display_GUI()

Setting value of url to https://www.gettinenglish.com/czytanki-po-angielsku-b1/


Output(layout=Layout(border='1px solid black'))

In [18]:
['a']*4

['a', 'a', 'a', 'a']